In [165]:
import pandas as pd

In [166]:
# Нужно рассчитать retention rate по месяцам для когорт пользователей, 
# где когорта определяется месяцем регистрации пользователя.

# Retention rate для месяца M после регистрации 
# - это доля пользователей когорты, у которых была хотя бы одна сессия 
# в M-том месяце после месяца регистрации.

# Результат должен быть в виде таблицы, где:
# - строки - когорты (пользователи по месяцу регистрации Х)
# - столбцы - месяцы после регистрации (от 0 до 11, где 0 - месяц регистрации)
# - значения - retention rate в процентах (сколько пользователей было в когорте и сколько вернулись)

# Алгоритм работы с когортными задачами
# 0. Формилируем задачу
# 1. Определяем когорту (событие и временной промежуток) - пользователи по месяцу регистрации Х
# 2. Выделяем целевые метрики - retention rate. Какой % пользователей, кто зарегистрировался в месяц Х продолжать приходить дальше с 0 по 11 месяца
# 3. Выбираем подходящий формат отчёта - возрастной 

# Что делаем: 
# 1. узнаём сколько пользователей и в какие месяца зарегистрировались 
# → Чтобы узнать месяц регистрации пользователей, мы используем таблицу users, используем колонку user_registration_dt в дату и затем в месяц и сохраняем в новую колонку
# → Чтобы узнать количество пользователей зарегистрированных в месяц Х, мы группируем этих пользователей по месяцам и затем считаем уникальные значения
# 2. узнаём сколько зарегистрированных пользователей возвращалось после даты регистрации
# → Мэтчу сессии с юзерами
# → Узнаю какие сессий у юзеров были после даты регистрации 
# → Считаю сессии по когортам с 0 по 11 месяц

In [167]:
# узнал, какой был месяц регистрации пользователя
users = pd.read_csv('/Users/artempak_macbook_air/Desktop/Code/python_learning/alex_python_for_pm/module_10_cohorts/self_tasks/data/users.csv', index_col='client_id')

users['user_registration_dt'] = pd.to_datetime(users['user_registration_dt'])
users['user_registration_month'] = users['user_registration_dt'].dt.to_period('M')

# сколько было юзеров зарегано в месяц создания когорты
cohort_size_users = users.groupby('user_registration_month').agg(u_users = ('user_id', 'count'))
cohort_size_users.head(2)

,u_users
user_registration_month,
2020-01,34
2020-02,38


In [168]:
# загружаю сессии
sessions = pd.read_csv('/Users/artempak_macbook_air/Desktop/Code/python_learning/alex_python_for_pm/module_10_cohorts/self_tasks/data/sessions.csv', index_col='client_id')

# делаю корректные даты сессий
sessions['session_start_ts'] = pd.to_datetime(sessions['session_start_ts'])
sessions['session_start_month'] = sessions['session_start_ts'].dt.to_period('M')
sessions.head(2)

,session_id,session_start_ts,session_end_ts,session_duration,utm_source,utm_campaign,utm_medium,utm_term,utm_content,session_start_month
client_id,,,,,,,,,,
1225900370-984223083,19d506f4-5ae7-11eb-b0b2-acde48001122,2020-07-22 00:06:48.141107,2020-07-22 00:07:46.208174,58,yandex_direct,gnrl|845242,gnrl,NaN,NaN,2020-07
1265117401-301250669,19d50a0a-5ae7-11eb-b0b2-acde48001122,2020-05-15 18:58:16.150037,2020-05-15 18:59:13.160074,57,fb,lal|772023,lal,NaN,NaN,2020-05


In [169]:
# связываю пользователей и сессии по client id, фильтрую сессии от по дате регистрации
cohorts_sessions = users.merge(sessions, how='inner', on='client_id')
cohorts_sessions = cohorts_sessions.query("session_start_month >= user_registration_month")
cohorts_sessions.head(2)

,user_id,user_email,user_registration_dt,user_city,user_registration_month,session_id,session_start_ts,session_end_ts,session_duration,utm_source,utm_campaign,utm_medium,utm_term,utm_content,session_start_month
client_id,,,,,,,,,,,,,,,
1120163048-362459827,1425,anilidic1822@outlook.com,2020-06-23,Екатеринбург,2020-06,19d51842-5ae7-11eb-b0b2-acde48001122,2020-06-23 10:54:25.161018,2020-06-23 10:55:18.223054,53,yandex_direct,gnrl|953233,gnrl,NaN,NaN,2020-06
830333202-483914880,1429,bornagain1847@gmail.com,2020-08-15,Санкт-Петербург,2020-08,19d5385e-5ae7-11eb-b0b2-acde48001122,2020-08-06 17:52:15.184076,2020-08-06 17:53:14.191138,59,yandex_direct,gnrl|445275,gnrl,NaN,NaN,2020-08


In [170]:
# считаю жизнь когорт в месяцах
cohorts_sessions['cohort_age_days'] = (cohorts_sessions['session_start_ts'] - cohorts_sessions['user_registration_dt']).dt.days
cohorts_sessions['cohort_age_month'] = cohorts_sessions['cohort_age_days'] // 30
cohorts_sessions.tail(2)

# cohorts_sessions.dtypes


,user_id,user_email,user_registration_dt,user_city,user_registration_month,session_id,session_start_ts,session_end_ts,session_duration,utm_source,utm_campaign,utm_medium,utm_term,utm_content,session_start_month,cohort_age_days,cohort_age_month
client_id,,,,,,,,,,,,,,,,,
817367844-160697484,3125,abridged1849@live.com,2020-07-18,Казань,2020-07,1a7a8c6e-5ae7-11eb-b0b2-acde48001122,2020-07-23 14:45:42.246332,2020-07-23 14:46:38.272415,56,yandex_direct,gnrl|326308,gnrl,NaN,NaN,2020-07,5,0
817367844-160697484,3125,abridged1849@live.com,2020-07-18,Казань,2020-07,1a7a8d90-5ae7-11eb-b0b2-acde48001122,2020-07-24 14:19:32.248388,2020-07-24 14:20:20.267448,48,NaN,NaN,NaN,NaN,NaN,2020-07,6,0
